In [1]:
import torch

In [2]:
words = open('names.txt' , 'r').read().splitlines()

In [3]:
t = {}
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1 , ch2 , ch3 in zip(chs , chs[1:] , chs[2:]):
        trigram = (ch1 , ch2 , ch3)
        t[trigram] = t.get(trigram , 0 )+1

In [4]:
(sorted(t.items(), key=lambda kv : -kv[1]))
len(t)

6037

In [5]:
N = torch.zeros((27,27 , 27) , dtype=torch.int32)

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [7]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1 , ch2 , ch3 in zip(chs , chs[1:] , chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[(ix1 , ix2) , ix3] += 1

In [10]:
p = N[0,1].float()
p /= p.sum()
p

tensor([0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370,
        0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370, 0.0370])

In [9]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p , num_samples=1 , replacement=True , generator=g).item()
itos[ix]

'c'

In [ ]:
P = (N).float()
P /= P.sum(1 , keepdim=True)
P.shape

torch.Size([27, 27, 27])

In [17]:
g = torch.Generator().manual_seed(2147483647)
for i in range(20):
    out = []
    ix = 0
    ix2=0
    while True:
        p = P[ix,ix2]
        ix3 = torch.multinomial(p , num_samples = 1 , replacement=True , generator = g).item()
        out.append(itos[ix3])
        if ix3==0:
            break

        ix1 , ix2 = ix2 , ix3

    print(''.join(out))

cexzm.
zoglkurkicqzktyhwmvmzimjttainrlkfukzkktda.
sfcxvpubjtbhrmgotzx.
iczixqctvujkwptedogkkjemkmmsidguenkbvgynywftbspmhwcivgbvtahlvsu.
dsdxxblnwglhpyiw.
igwnjwrpfdwipkwzkm.
desu.
firmt.
gbiksjbquabsvoth.
kuysxqevhcmrbxmcwyhrrjenvxmvpfkmwmghfvjzxobomysox.
gbptjapxweegpfwhccfyzfvksiiqmvwbhmiwqmdgzqsamjhgamcxwmmk.
iswcxfmbalcslhy.
fpycvasvz.
bqzazeunschck.
wnkojuoxyvtvfiwksddugnkul.
fuwfcgjz.
abl.
j.
nuuutstofgqzubbo.
rdubpknhmd.


In [18]:
for w in words[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2,ch3 in zip(chs , chs[1:] , chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        prob = P[ix1 , ix2 , ix3]
        print(f'{ch1}{ch2}{ch3}: {prob:.4f}')

.em: 0.1276
emm: 0.0294
mma: 0.0160
ma.: 0.2269
.ol: 0.0811
oli: 0.0560
liv: 0.1278
ivi: 0.0102
via: 0.1896
ia.: 0.1558
.av: 0.3143
ava: 0.0187
va.: 0.1747


In [19]:
logprob = 0
n=0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2,ch3 in zip(chs , chs[1:] , chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        prob = P[ix1 , ix2 , ix3]
        n+=1
        logprob += torch.log(prob)

print(f'{logprob:.4f}')
neg_log = -logprob
print(f'{neg_log/n :.4f}')


-564270.9375
2.8773


**Neural Network Approach**

In [90]:
#creating a dataset
xs , ys = [] , []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2,ch3 in zip(chs , chs[1:] , chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append([ix1,ix2])
        ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = len(xs)
print("no of examples: " , num)

no of examples:  196113


In [91]:
xs

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        ...,
        [26, 25],
        [25, 26],
        [26, 24]])

In [92]:
#1 Hot Encoding 
import torch.nn.functional as F


In [93]:
xenc = F.one_hot(xs , num_classes=27).float()   
xenc = xenc.view(xs.shape[0], -1)
W = torch.randn((54,27))                         
xenc @ W
logits = xenc @ W 
counts = logits.exp()
prob = counts / counts.sum(1 , keepdims=True)             
          

In [94]:
nlls = torch.zeros(5)

for i in range(5):
  # i-th bigram:
  x = xs[i] # input character index
  y = ys[i].item() # label character index
  ix1 = x[0].item()
  ix2 = x[1].item()
  print('--------')
  print(f'bigram example {i+1}: {itos[ix1]}{itos[ix2]}{itos[y]} (indexes {ix1}{ix2},{y})')
  print('input to the neural net:', x.tolist(   ))
  print('output probabilities from the neural net:', prob[i])
  print('label (actual next character):', y)
  p = prob[i,y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: .em (indexes 05,13)
input to the neural net: [0, 5]
output probabilities from the neural net: tensor([0.0016, 0.1472, 0.0121, 0.0096, 0.1285, 0.0216, 0.0095, 0.0788, 0.0063,
        0.0224, 0.0106, 0.0148, 0.0019, 0.0141, 0.0104, 0.0922, 0.0067, 0.0058,
        0.0050, 0.1280, 0.0051, 0.0075, 0.0375, 0.1142, 0.0035, 0.0246, 0.0805])
label (actual next character): 13
probability assigned by the net to the the correct character: 0.014113913290202618
log likelihood: -4.260594367980957
negative log likelihood: 4.260594367980957
--------
bigram example 2: emm (indexes 513,13)
input to the neural net: [5, 13]
output probabilities from the neural net: tensor([0.0039, 0.0156, 0.0065, 0.0262, 0.0155, 0.1858, 0.0100, 0.0530, 0.0888,
        0.0231, 0.0093, 0.0098, 0.0081, 0.0151, 0.0166, 0.0030, 0.0468, 0.0135,
        0.0151, 0.0936, 0.0121, 0.0014, 0.0081, 0.0075, 0.2752, 0.0262, 0.0102])
label (actual next character): 13
probability assigned by the net to the the co

In [95]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54,27) , generator=g , requires_grad=True)
xenc = F.one_hot(xs , num_classes=27).float()
xenc = xenc.view(xs.shape[0], -1)
logits = xenc @ W
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(n) , ys].log().mean()
print(loss.item())

4.186270713806152


In [97]:
for k in range(100):
    xenc = F.one_hot(xs , num_classes=27).float()
    xenc = xenc.view(xs.shape[0], -1)
    logits = xenc @ W
    counts = logits.exp()
    prob = counts / counts.sum(1 , keepdims = True)
    loss = -prob[torch.arange(num) , ys].log().mean() 
    print(loss.item())
    W.grad = None
    loss.backward()
    W.data += -50.0*W.grad

2.3821403980255127
2.375330686569214
2.3691024780273438
2.3633830547332764
2.358112096786499
2.353239059448242
2.348720073699951
2.3445184230804443
2.3406026363372803
2.336944818496704
2.3335204124450684
2.3303089141845703
2.327291250228882
2.324450969696045
2.3217737674713135
2.3192460536956787
2.316856622695923
2.314594030380249
2.3124499320983887
2.310415029525757
2.3084816932678223
2.3066422939300537
2.3048911094665527
2.3032217025756836
2.301628828048706
2.300107002258301
2.2986526489257812
2.2972605228424072
2.2959272861480713
2.294649362564087
2.2934229373931885
2.292245388031006
2.291114091873169
2.2900257110595703
2.288978099822998
2.2879693508148193
2.286997079849243
2.2860591411590576
2.285153865814209
2.2842795848846436
2.2834348678588867
2.2826180458068848
2.281827688217163
2.281062602996826
2.2803215980529785
2.2796032428741455
2.278907060623169
2.278231382369995
2.277575969696045
2.2769391536712646
2.276320695877075
2.27571964263916
2.275135040283203
2.274566411972046
2.

In [104]:
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    ix1 = 0
    ix2 = 0
    while True:
        x = torch.tensor([ix1,ix2])
        xenc = F.one_hot( x , num_classes=27).float()
        xenc = xenc.view(1, 54)
        logits = xenc@W
        counts = logits.exp()
        p = counts / counts.sum(1 , keepdim=True)
        ix3 = torch.multinomial(p , num_samples=1 , replacement=True , generator=g).item()
        out.append(itos[ix3])
        if ix3==0:
            break
        ix1 , ix2 = ix2 , ix3

    print(''.join(out))

ae.
za.
ahallurailaziaydamellimittain.
alayn.
ka.
